TED talk text generator

Generate using a character based model.

The code is this notebook is a partial modification of several tutorials, blog posts and YouTube videos. The main sources include:
* [Text Generation With LSTM Recurrent Neural Networks in Python with Keras](https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/) by Dr. Jason Brownlee
* [Text Generation with Python and TensorFlow/Keras](https://stackabuse.com/text-generation-with-python-and-tensorflow-keras/) by Dan Nelson
* YouTube: [167 - Text prediction using LSTM (English text)](https://youtu.be/zyCpntcVKSo?t=2) by Sreenivas B.


In [1]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.train import latest_checkpoint
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, LSTM
import random
import textwrap

In [2]:
# Attach to my Google drive so I can save the csv file later
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
#tag_name = 'all_transcripts'
#tag_name = 'brain'
#tag_name = 'business'
#tag_name = 'creativity'
#tag_name = 'culture'
#tag_name = 'psychology'
tag_name = 'science'

lower_corpus = False

if lower_corpus:
    corpus_file_name = tag_name + '_lowercase'
else:
    corpus_file_name = tag_name + '_sentence_case'

# This switch makes it easier running locally or in a Google Colab environment
colab = False

if colab:
    root_path = '/content/drive/MyDrive/ted/'
else:
    root_path = '../data/'
    
sequence_length = 50

In [4]:
with open(f'{root_path}{corpus_file_name}.txt', encoding='utf-8') as f:
    corpus = f.read()
f.close()

In [5]:
corpus[0:500]

"So I want to start by offering you a free no-tech life hack, and all it requires of you is  that you change your posture for two minutes. But before I give it away, I want to ask you to right now do a little audit of your body and what you're doing with your body. So how many of you are sort of making yourselves smaller? Maybe you're hunching, crossing your legs, maybe wrapping your ankles. Sometimes we hold onto our arms like this. Sometimes we spread out.  I see you. So I want you to pay atten"

The code below is used in almost every article I have seen about LSTM for text generation, but this snippet is adapted from :
https://www.kaggle.com/mrisdal/intro-to-lstms-w-keras-gpu-for-text-generation/

In [6]:
all_chars = sorted(list(set(corpus)))

#remove most punctuation, but leave spaces, exclamation points, and periods
list_to_remove = ['"', '&', '-', ',', "'", '/', ';',  '—']

chars = [character for character in all_chars if character not in list_to_remove]
# print(chars)
# # hat tip to this post for the idea of removing one list from another
# # https://www.geeksforgeeks.org/python-remove-all-values-from-a-list-present-in-other-list/

In [7]:
# remove all unwanted characters from the original corpus
corpus_list = [character for character in corpus if character not in list_to_remove]

# Re-join all letters back into a single string as the updated corpus
corpus = ''.join(corpus_list)

In [8]:
# If running models with punctuation removed, leave commented out
# chars = sorted(list(set(corpus)))

print('Number of unique characters in the corpus:', len(chars))

char_indices = dict((c, i) for i, c in enumerate(chars))

indices_char = dict((i, c) for i, c in enumerate(chars))

Number of unique characters in the corpus: 68


In [9]:
# Show character to index mapping
print (char_indices)

{' ': 0, '!': 1, '%': 2, '.': 3, '0': 4, '1': 5, '2': 6, '3': 7, '4': 8, '5': 9, '6': 10, '7': 11, '8': 12, '9': 13, '?': 14, 'A': 15, 'B': 16, 'C': 17, 'D': 18, 'E': 19, 'F': 20, 'G': 21, 'H': 22, 'I': 23, 'J': 24, 'K': 25, 'L': 26, 'M': 27, 'N': 28, 'O': 29, 'P': 30, 'Q': 31, 'R': 32, 'S': 33, 'T': 34, 'U': 35, 'V': 36, 'W': 37, 'X': 38, 'Y': 39, '[': 40, ']': 41, 'a': 42, 'b': 43, 'c': 44, 'd': 45, 'e': 46, 'f': 47, 'g': 48, 'h': 49, 'i': 50, 'j': 51, 'k': 52, 'l': 53, 'm': 54, 'n': 55, 'o': 56, 'p': 57, 'q': 58, 'r': 59, 's': 60, 't': 61, 'u': 62, 'v': 63, 'w': 64, 'x': 65, 'y': 66, 'z': 67}


In [10]:
# Show index to character mapping
print(indices_char)

{0: ' ', 1: '!', 2: '%', 3: '.', 4: '0', 5: '1', 6: '2', 7: '3', 8: '4', 9: '5', 10: '6', 11: '7', 12: '8', 13: '9', 14: '?', 15: 'A', 16: 'B', 17: 'C', 18: 'D', 19: 'E', 20: 'F', 21: 'G', 22: 'H', 23: 'I', 24: 'J', 25: 'K', 26: 'L', 27: 'M', 28: 'N', 29: 'O', 30: 'P', 31: 'Q', 32: 'R', 33: 'S', 34: 'T', 35: 'U', 36: 'V', 37: 'W', 38: 'X', 39: 'Y', 40: '[', 41: ']', 42: 'a', 43: 'b', 44: 'c', 45: 'd', 46: 'e', 47: 'f', 48: 'g', 49: 'h', 50: 'i', 51: 'j', 52: 'k', 53: 'l', 54: 'm', 55: 'n', 56: 'o', 57: 'p', 58: 'q', 59: 'r', 60: 's', 61: 't', 62: 'u', 63: 'v', 64: 'w', 65: 'x', 66: 'y', 67: 'z'}


In [11]:
print (f'The input corpus contains {len(corpus):,} characters.')

The input corpus contains 78,510 characters.


The model will use arbitrary length of characters, e.g. 40, and then predict the next character that will appear (the 41st).

I would like to change this sequence length in different models, to see how it might affect the model's performance.

Larger might be better?

Loop through the corpus, creating segments of 40 characters, plus a segment of a single character that would appear after it. These segments are then converted into their respective digits, and loaded into X

In addition, create a list of target values. The target, y, is a single text character, and is also converted into a corresponding numeric value.

In [12]:
corpus[0:50]

'So I want to start by offering you a free notech l'

In [13]:
corpus[1:41]

'o I want to start by offering you a free'

In [14]:
corpus[41]

' '

In [15]:
# For future modeling, we could try to offset the sequences by more than one
# letter. For now the model will look at every sequence right after another.
step = 1

X_numeric_list = []
y_numeric_list = []


for i in range (0, len(corpus) - sequence_length, step):
    # To find X when still as characters, loop through and extract
    # a sequence for example, from the 2nd to the 42nd, as in corpus[1:41] since 
    # the string index is zero-based
    X_char_sequence = corpus[i:i + sequence_length]  #exclusive
    y_char = corpus[i + sequence_length]

    # Convert the X character sequence into a list of integers, using the 
    # dictionary created above.
    X_numeric_list.append( [char_indices[letter] for letter in X_char_sequence])
    # Also convert target letter y to it's corresponding numeric value in the
    # dictionary
    y_numeric_list.append(char_indices[y_char])



In [16]:
# Look at the first two converted sequences
print ( X_numeric_list[0])
print ( X_numeric_list[1])


[33, 56, 0, 23, 0, 64, 42, 55, 61, 0, 61, 56, 0, 60, 61, 42, 59, 61, 0, 43, 66, 0, 56, 47, 47, 46, 59, 50, 55, 48, 0, 66, 56, 62, 0, 42, 0, 47, 59, 46, 46, 0, 55, 56, 61, 46, 44, 49, 0, 53]
[56, 0, 23, 0, 64, 42, 55, 61, 0, 61, 56, 0, 60, 61, 42, 59, 61, 0, 43, 66, 0, 56, 47, 47, 46, 59, 50, 55, 48, 0, 66, 56, 62, 0, 42, 0, 47, 59, 46, 46, 0, 55, 56, 61, 46, 44, 49, 0, 53, 50]


In [17]:
print ( y_numeric_list[0:2])

[50, 47]


In [18]:
indices_char[51]

'j'

In [19]:
corpus[0:43]

'So I want to start by offering you a free n'

This example is a little hard to follow, since we happened to be in the middle of the word 'been'. The letter e is mapped to 51.

However the beginning of the sequence has been shifted by one as well.

In [20]:
print (f'There are {len(X_numeric_list):,} entries in X_numeric_list') 

There are 78,460 entries in X_numeric_list


An LSTM model needs the X data to be in 3 dimensions:
* Samples (number of rows)
* Time steps (this implies time series. In this example this corresponds to the sequence_length)
* Features (the target, or y variable)

This [blog post by Dr. John Brownlee](https://machinelearningmastery.com/reshape-input-data-long-short-term-memory-networks-keras/) contains a longer explanation, which I paraphrased here.

In [21]:
# reshape X to be [samples, time steps, features]
# Samples is the total number of 40 character text fragments created above
# Time step is the sequence length, which is this case is 40, but might change
# in other model variations
# Feature is one, since the model is predicting one character at a time
X = np.reshape(X_numeric_list, (len(X_numeric_list), sequence_length, 1))

In [22]:
X.shape

(78460, 50, 1)

In [23]:
# normalize/scale the data by dividing by the length of the character list.
X = X / len(chars)

In [24]:
# Look at the first value of X
#X[0]

Even thought the values for the target variable are integers, they are essentially labels for the predicted characters. They are one-hot encoded using the to_categorical function.

In [25]:
y = to_categorical(y_numeric_list)
y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

---
### Load LSTM Model
In this notebook, load a model that was previously trained and saved.

In [26]:
# Save the model to my Google Drive so I can load it later from another notebook
# model = load_model(f'/content/drive/MyDrive/ted/models/ted_model_{tag_name}')

# model.summary()

If a model failed during fitting, we can still load it's most recent weights which were stored in a checkpoint file.

Use of ModelCheckPoint to save the model's progress example as described in the [TensorFlow documentation.](https://www.tensorflow.org/tutorials/keras/save_and_load#checkpoint_callback_options)

In [27]:
checkpoint_dir = f'{root_path}{corpus_file_name}/'

latest = latest_checkpoint(checkpoint_dir)


In [28]:
# On Colab, model saved as 
# model.save(f'/content/drive/MyDrive/ted/models/ted_model_{tag_name}_seq_{sequence_length}')

In [29]:
print ( f'{root_path}models/ted_model_{tag_name}')

../data/models/ted_model_science


In [30]:
# define the LSTM model
# model = Sequential()
# model.add(LSTM(1000, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(256))
# model.add(Dropout(0.2))
# model.add(Dense(y.shape[1], activation='softmax'))


# # Load the previously saved weights
# model.load_weights(latest)

# model.compile(loss='categorical_crossentropy', optimizer='adam' , metrics = ['accuracy','Recall','Precision'])
#model = load_model(f'{latest}')

# Use if there was a successful save of a model


#model = load_model(f'{root_path}models/ted_model_{tag_name}')
model = load_model(f'{root_path}/models/ted_model_{tag_name}')

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 50, 1000)          4008000   
_________________________________________________________________
dropout_2 (Dropout)          (None, 50, 1000)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               1287168   
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 68)                17476     
Total params: 5,312,644
Trainable params: 5,312,644
Non-trainable params: 0
_________________________________________________________________


X_numeric_list contains all of our 40 character sequences. Choose one for a seed for the text generation.

In [31]:
seed_start = random.randint(0, len(corpus) - sequence_length)
print(f'Random start index for seed text: {seed_start} sequence.')

X_seeded = X_numeric_list[seed_start]
print(X_seeded)

Random start index for seed text: 8050 sequence.
[46, 59, 46, 55, 44, 46, 60, 0, 56, 55, 0, 61, 64, 56, 0, 52, 46, 66, 0, 0, 61, 46, 60, 61, 56, 60, 61, 46, 59, 56, 55, 46, 0, 64, 49, 50, 44, 49, 0, 50, 60, 0, 61, 49, 46, 0, 45, 56, 54, 50]


Convert this back to text so we can see what it looks like.

In [32]:
seed_string = ''.join([indices_char[letter_code] for letter_code in X_seeded])
seed_string

'erences on two key  testosterone which is the domi'

To generate text, we need to change the input text into a 3D shape as we did the X input data.

In [33]:
temp_x = np.reshape(X_seeded, (1, len(X_seeded) , 1))
# normalize/scale the data by dividing by the length of the character list.
temp_x = temp_x / len(chars)
prediction = model.predict(temp_x, verbose=0)
print(f"A prediction's shape:{prediction.shape}")
print('Prediction:')
print (prediction)

A prediction's shape:(1, 68)
Prediction:
[[9.01670865e-06 1.67315932e-07 7.84596243e-16 1.63557644e-07
  2.35479686e-16 3.51622342e-11 3.95659796e-13 3.46711380e-15
  3.55786228e-14 4.32721052e-13 3.03968448e-11 6.13680417e-12
  1.44444292e-11 4.72688837e-14 1.07734891e-07 1.12283681e-11
  8.71870528e-14 1.75224622e-14 1.06849328e-12 8.93032610e-12
  2.57501043e-09 4.03863087e-13 6.57680697e-14 2.65548954e-12
  3.69645183e-11 1.20949792e-10 4.56405096e-13 1.45210358e-10
  6.96408399e-14 5.03313675e-11 1.32617929e-13 7.35243487e-15
  7.35283882e-12 1.51552833e-11 3.72451072e-12 2.88187443e-12
  4.11755509e-13 2.21744689e-11 1.06282899e-15 2.25444576e-12
  2.45700104e-18 9.86975224e-11 2.75793311e-04 1.67417238e-04
  7.56032299e-03 6.77888820e-05 2.39993708e-04 7.03381229e-05
  4.10826993e-04 1.38482192e-05 6.16201123e-06 2.06102141e-06
  1.00086289e-07 5.96936559e-04 4.13715388e-05 9.89724994e-01
  9.48044544e-06 9.31184695e-05 2.87266062e-06 1.06017405e-05
  4.17593197e-04 8.96501952e-

Each prediction produces an result of shape (1, 75), and contains a predicted probability of a character. Use argmax to find the index of the highest probability.

In [34]:
np.argmax(prediction)

55

So the first letter predicted after the seed sequence is:

In [35]:
indices_char[np.argmax(prediction)]

'n'

In [36]:
# Put all predicted characters in a list, then convert to a string later.
complete_predicted_text =[]

# Generate 1000 characters
for i in range(600):
    # reshape to 1 row, sequence length, and 1 for predicting one character as the target
    temp_x = np.reshape(X_seeded, (1, len(X_seeded) , 1))
    # normalize/scale the data by dividing by the length of the character list.
    temp_x = temp_x / len(chars)
    prediction = model.predict(temp_x, verbose=0)
    complete_predicted_text.append(indices_char[np.argmax(prediction)])

    # After predicting, add this character's numeric value on to the randomly
    # chose seed string, and use this for another prediction
    X_seeded.append(np.argmax(prediction))

    # Move the seeded text up one character, so create a new sequence
    X_seeded = X_seeded[1:len(X_seeded)]


#print (complete_predicted_text)


The predicted results are a list of characters, so convert into a string

In [37]:
predicted_string = ''.join(complete_predicted_text)

# Wrap to 120 characters just so I can visually compare to the original string.
textwrap.wrap(seed_string + '->'+ predicted_string, width=120)

['erences on two key  testosterone which is the domi->nance hormone and sorted mn the byternal world. So what we tand to',
 'do in onre of the sout of eerice. Its woree ieed sooeont werk been stuty  So we though tuamected inow this widt they do',
 'tpter when toue and they do it what they do it what you geel powerful  So I gott its not possible. Than you dan be',
 'sialking to do in the sersone besuer wese been tale aaautaclly donlaated in and of them  Then you went on your',
 'bodin.anonhgtie youre martioy. So this is what were toot of souer upenis silet? And I lave aboogs the pie have',
 'interisted  I dont maa ang mystefies byeepinns ard alled tiat when toent hav']

In [38]:
# Find the original seed string to see what came after it
original_paragraph = corpus[corpus.find(seed_string): corpus.find(seed_string) + 2000]

textwrap.wrap(original_paragraph, width=120)

['erences on two key  testosterone which is the dominance hormone and cortisol which is the stress hormone. So what we',
 'find is that highpower alpha males in primate hierarchies have high testosterone and low cortisol and powerful and',
 'effective leaders also have high testosterone and low cortisol. So what does that mean? When you think about power',
 'people tended to think only about testosterone because that was about dominance. But really power is also about how you',
 'react to stress. So do you want the highpower leader thats dominant high on testosterone but really stress reactive?',
 'Probably not right? You want the person whos powerful and assertive and dominant but not very stress reactive the person',
 'whos laid back. So we know that in primate hierarchies if an alpha needs to take over if an individual needs to take',
 'over an alpha role sort of suddenly within a few days that individuals testosterone has gone up significantly and his',
 'cortisol has dropped signi